# Beyond Bluetooth with alternate encoding (Random Forest)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
cd ..

C:\Users\mtom7\Documents\_PhD\Bootcamp\Group Project\ml-labs-bootcamp-group-5


## Make Data 

In [4]:
from src.features.beyond import sensor_summary
from src.features.common import postproc_default
from src.dataset.tc4tl import make_datasets

In [4]:
(trainset, devset, testset), pipe = make_datasets(sensor_summary, postproc_default, verbose=True,
                                          testing=0, tunables={})

100%|██████████████████████████████████████████████████████████████████████████████| 8423/8423 [10:05<00:00, 13.90it/s]


In [5]:
trainset

,Bluetooth:Min,Bluetooth:Percentile_1,Bluetooth:Percentile_5,Bluetooth:Percentile_10,Bluetooth:Percentile_25,Bluetooth:Percentile_40,Bluetooth:Mean,Bluetooth:Percentile_65,Bluetooth:Percentile_80,Bluetooth:Percentile_90,...,Heading_y2:Min,Heading_y2:Mean,Heading_y2:Max,Heading_z2:Min,Heading_z2:Mean,Heading_z2:Max,CoarseGrain,fileid,Distance,DistanceFloat
0,-88.0,-79.00,-75.0,-74.0,-67.00,-64.0,-62.133482,-59.0,-54.0,-52.0,...,-54.842712,-43.990857,-39.718319,-36.270050,-30.593178,-14.488922,0,aaadbuvp_tc4tl20.csv,1.8,1.8
1,-76.0,-76.00,-64.0,-60.0,-57.00,-56.0,-55.255556,-54.0,-50.0,-49.0,...,22.951851,25.961174,38.456741,-32.853706,31.900434,44.096672,1,aaamkcii_tc4tl20.csv,3.0,3.0
2,-89.0,-80.00,-76.6,-75.0,-69.00,-63.0,-62.289089,-57.0,-54.0,-52.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,aabqtowt_tc4tl20.csv,1.2,1.2
3,-93.0,-87.00,-83.0,-78.0,-71.00,-68.0,-67.938259,-64.0,-62.0,-61.0,...,-35.223030,-24.979674,-14.252491,-39.986511,-28.141121,-6.259827,1,aadkjwss_tc4tl20.csv,1.8,1.8
4,-72.0,-70.00,-65.0,-63.0,-58.00,-57.0,-55.760797,-54.0,-51.0,-48.0,...,-34.928822,-19.842537,9.745598,-49.795044,-42.289961,-36.626099,0,aafzrgzt_tc4tl20.csv,1.8,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,-87.0,-84.00,-77.0,-74.0,-67.00,-65.0,-62.998435,-58.3,-54.0,-52.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,zzwcoemu_tc4tl20.csv,1.2,1.2
15548,-93.0,-92.00,-89.0,-86.0,-81.25,-78.0,-77.020677,-74.0,-71.0,-69.0,...,-8.760864,-6.220505,-2.499344,-43.627808,-40.783590,-38.693420,1,zzwoqxuc_tc4tl20.csv,4.5,4.5
15549,-99.0,-94.00,-88.0,-85.0,-80.00,-77.0,-74.469638,-71.0,-65.0,-63.0,...,42.117676,46.289325,49.303135,-23.620674,4.104991,20.750044,1,zzymwbbb_tc4tl20.csv,1.2,1.2
15550,-88.0,-83.39,-79.0,-75.0,-71.00,-68.0,-64.321346,-60.0,-56.0,-52.0,...,-18.655930,-7.594508,14.214676,-43.580078,-41.863241,-40.822937,0,zzzflikz_tc4tl20.csv,1.8,1.8


In [6]:
predictors = []
for col in trainset.columns:
    if "Rssi" in col or "Coarse" in col or "_" in col:
        predictors.append(col)
predictors

['Bluetooth:Percentile_1',
 'Bluetooth:Percentile_5',
 'Bluetooth:Percentile_10',
 'Bluetooth:Percentile_25',
 'Bluetooth:Percentile_40',
 'Bluetooth:Percentile_65',
 'Bluetooth:Percentile_80',
 'Bluetooth:Percentile_90',
 'Bluetooth:Percentile_95',
 'Bluetooth:Percentile_99',
 'Accelerometer_x:Min',
 'Accelerometer_x:Mean',
 'Accelerometer_x:Max',
 'Accelerometer_y:Min',
 'Accelerometer_y:Mean',
 'Accelerometer_y:Max',
 'Accelerometer_z:Min',
 'Accelerometer_z:Mean',
 'Accelerometer_z:Max',
 'Gyroscope_x:Min',
 'Gyroscope_x:Mean',
 'Gyroscope_x:Max',
 'Gyroscope_y:Min',
 'Gyroscope_y:Mean',
 'Gyroscope_y:Max',
 'Gyroscope_z:Min',
 'Gyroscope_z:Mean',
 'Gyroscope_z:Max',
 'Attitude_x:Min',
 'Attitude_x:Mean',
 'Attitude_x:Max',
 'Attitude_y:Min',
 'Attitude_y:Mean',
 'Attitude_y:Max',
 'Attitude_z:Min',
 'Attitude_z:Mean',
 'Attitude_z:Max',
 'Gravity_x:Min',
 'Gravity_x:Mean',
 'Gravity_x:Max',
 'Gravity_y:Min',
 'Gravity_y:Mean',
 'Gravity_y:Max',
 'Gravity_z:Min',
 'Gravity_z:Mean',

## Evaluate

In [7]:
from src.models.evaluate import dual_evaluation
import pandas as pd
from src.ndcf import compute_ndcf

report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.09265216819522434

In [8]:
ndcf

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.086709,0.015744,0.102454
1,fine_grain,1.8,0.063588,0.019756,0.083344
2,fine_grain,3.0,0.034762,0.045032,0.079794
3,coarse_grain,1.8,0.085032,0.019986,0.105017


In [9]:
print(report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.95      0.91      0.93      1407
         1.8       0.93      0.89      0.91      1408
         3.0       0.92      0.95      0.93      1385
         4.5       0.90      0.95      0.93      1399

    accuracy                           0.92      5599
   macro avg       0.93      0.93      0.92      5599
weighted avg       0.93      0.92      0.92      5599



In [10]:
print(report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.98      0.91      0.95      1423
         4.5       0.92      0.98      0.95      1401

    accuracy                           0.95      2824
   macro avg       0.95      0.95      0.95      2824
weighted avg       0.95      0.95      0.95      2824



## Combine with Categorical Features

In [5]:
from src.features.common import postproc_categorical, read_non_sensor_data
from src.featutils import combine
import category_encoders as ce
from src.models.evaluate import dual_evaluation
import pandas as pd
from src.ndcf import compute_ndcf

In [10]:
def run_with_encoder(encoder):
    
    (trainset, devset, testset), pipe = make_datasets(combine(sensor_summary, read_non_sensor_data),
                                          postproc_categorical, verbose=True, testing=0,
                                          tunables={ 
                                              "CategoricalEncoder" : encoder
                                          })
    
    predictors = []
    for col in trainset.columns:
        if "Rssi" in col or "Coarse" in col or "_" in col:
            predictors.append(col)
            
    report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                        predictors=predictors,
                        target="Distance")
    ndcf = compute_ndcf(sysout)
    print(ndcf["nDCF"].mean())
    print(ndcf)
    print(report["cg=1"])
    print(report["cg=0"])

## BackwardDifferenceEncoder

In [11]:
(trainset, devset, testset), pipe = make_datasets(combine(sensor_summary, read_non_sensor_data),
                                          postproc_categorical, verbose=True, testing=0,
                                          tunables={ 
                                              "CategoricalEncoder" : ce.BackwardDifferenceEncoder
                                          })

100%|████████████████████████████████████████████████████████████████████████████| 15552/15552 [14:12<00:00, 18.24it/s]


NO Pipe input given!


C:\Users\mtom7\anaconda3\envs\py39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
100%|████████████████████████████████████████████████████████████████████████████████| 935/935 [00:59<00:00, 15.70it/s]


Pipe input given!


100%|██████████████████████████████████████████████████████████████████████████████| 8423/8423 [08:03<00:00, 17.44it/s]


Pipe input given!


In [12]:
trainset

,Bluetooth:Min,Bluetooth:Percentile_1,Bluetooth:Percentile_5,Bluetooth:Percentile_10,Bluetooth:Percentile_25,Bluetooth:Percentile_40,Bluetooth:Mean,Bluetooth:Percentile_65,Bluetooth:Percentile_80,Bluetooth:Percentile_90,...,Cat:RXDevice_3,Cat:RXDevice_4,Cat:RXDevice_5,Cat:RXDevice_6,Cat:RXDevice_7,Cat:RXDevice_8,Cat:TXCarry_0,Cat:RXCarry_0,Cat:RXPose_0,Cat:TXPose_0
0,-88.0,-79.00,-75.0,-74.0,-67.00,-64.0,-62.133482,-59.0,-54.0,-52.0,...,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,-0.5,-0.5,-0.5,-0.5
1,-76.0,-76.00,-64.0,-60.0,-57.00,-56.0,-55.255556,-54.0,-50.0,-49.0,...,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,-0.5,0.5,-0.5,-0.5
2,-89.0,-80.00,-76.6,-75.0,-69.00,-63.0,-62.289089,-57.0,-54.0,-52.0,...,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0.5,-0.5,-0.5,-0.5
3,-93.0,-87.00,-83.0,-78.0,-71.00,-68.0,-67.938259,-64.0,-62.0,-61.0,...,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,-0.5,0.5,0.5,0.5
4,-72.0,-70.00,-65.0,-63.0,-58.00,-57.0,-55.760797,-54.0,-51.0,-48.0,...,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0.5,-0.5,-0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,-87.0,-84.00,-77.0,-74.0,-67.00,-65.0,-62.998435,-58.3,-54.0,-52.0,...,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,-0.5,-0.5,-0.5,-0.5
15548,-93.0,-92.00,-89.0,-86.0,-81.25,-78.0,-77.020677,-74.0,-71.0,-69.0,...,0.4,0.5,-0.4,-0.3,-0.2,-0.1,0.5,-0.5,0.5,-0.5
15549,-99.0,-94.00,-88.0,-85.0,-80.00,-77.0,-74.469638,-71.0,-65.0,-63.0,...,0.4,0.5,0.6,-0.3,-0.2,-0.1,-0.5,0.5,-0.5,0.5
15550,-88.0,-83.39,-79.0,-75.0,-71.00,-68.0,-64.321346,-60.0,-56.0,-52.0,...,0.4,0.5,-0.4,-0.3,-0.2,-0.1,0.5,-0.5,0.5,-0.5


In [13]:
predictors = []
for col in trainset.columns:
    if "Rssi" in col or "Coarse" in col or "_" in col:
        predictors.append(col)
predictors

['Bluetooth:Percentile_1',
 'Bluetooth:Percentile_5',
 'Bluetooth:Percentile_10',
 'Bluetooth:Percentile_25',
 'Bluetooth:Percentile_40',
 'Bluetooth:Percentile_65',
 'Bluetooth:Percentile_80',
 'Bluetooth:Percentile_90',
 'Bluetooth:Percentile_95',
 'Bluetooth:Percentile_99',
 'Accelerometer_x:Min',
 'Accelerometer_x:Mean',
 'Accelerometer_x:Max',
 'Accelerometer_y:Min',
 'Accelerometer_y:Mean',
 'Accelerometer_y:Max',
 'Accelerometer_z:Min',
 'Accelerometer_z:Mean',
 'Accelerometer_z:Max',
 'Gyroscope_x:Min',
 'Gyroscope_x:Mean',
 'Gyroscope_x:Max',
 'Gyroscope_y:Min',
 'Gyroscope_y:Mean',
 'Gyroscope_y:Max',
 'Gyroscope_z:Min',
 'Gyroscope_z:Mean',
 'Gyroscope_z:Max',
 'Attitude_x:Min',
 'Attitude_x:Mean',
 'Attitude_x:Max',
 'Attitude_y:Min',
 'Attitude_y:Mean',
 'Attitude_y:Max',
 'Attitude_z:Min',
 'Attitude_z:Mean',
 'Attitude_z:Max',
 'Gravity_x:Min',
 'Gravity_x:Mean',
 'Gravity_x:Max',
 'Gravity_y:Min',
 'Gravity_y:Mean',
 'Gravity_y:Max',
 'Gravity_z:Min',
 'Gravity_z:Mean',

In [14]:
report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.09072033343095187

In [17]:
ndcf

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.083156,0.018130,0.101285
1,fine_grain,1.8,0.063588,0.016164,0.079752
2,fine_grain,3.0,0.036667,0.047177,0.083843
3,coarse_grain,1.8,0.077301,0.020700,0.098001


In [15]:
print(report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.94      0.92      0.93      1407
         1.8       0.95      0.88      0.92      1408
         3.0       0.92      0.95      0.94      1385
         4.5       0.90      0.95      0.92      1399

    accuracy                           0.93      5599
   macro avg       0.93      0.93      0.93      5599
weighted avg       0.93      0.93      0.93      5599



In [16]:
print(report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.98      0.92      0.95      1423
         4.5       0.93      0.98      0.95      1401

    accuracy                           0.95      2824
   macro avg       0.95      0.95      0.95      2824
weighted avg       0.95      0.95      0.95      2824



In [77]:
#trainset.to_csv('data/trainset.csv', index=False)
#testset.to_csv('data/testset.csv', index=False)
#devset.to_csv('data/devset.csv', index=False)

In [78]:
#trainset.shape

(15552, 123)

## BaseNEncoder

In [18]:
(trainset, devset, testset), pipe = make_datasets(combine(sensor_summary, read_non_sensor_data),
                                          postproc_categorical, verbose=True, testing=0,
                                          tunables={ 
                                              "CategoricalEncoder" : ce.BaseNEncoder
                                          })

100%|████████████████████████████████████████████████████████████████████████████| 15552/15552 [11:32<00:00, 22.44it/s]


NO Pipe input given!


C:\Users\mtom7\anaconda3\envs\py39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
100%|████████████████████████████████████████████████████████████████████████████████| 935/935 [01:00<00:00, 15.47it/s]


Pipe input given!


100%|██████████████████████████████████████████████████████████████████████████████| 8423/8423 [09:26<00:00, 14.87it/s]


Pipe input given!


In [19]:
trainset

,Bluetooth:Min,Bluetooth:Percentile_1,Bluetooth:Percentile_5,Bluetooth:Percentile_10,Bluetooth:Percentile_25,Bluetooth:Percentile_40,Bluetooth:Mean,Bluetooth:Percentile_65,Bluetooth:Percentile_80,Bluetooth:Percentile_90,...,Cat:RXDevice_3,Cat:RXDevice_4,Cat:TXCarry_0,Cat:TXCarry_1,Cat:RXCarry_0,Cat:RXCarry_1,Cat:RXPose_0,Cat:RXPose_1,Cat:TXPose_0,Cat:TXPose_1
0,-88.0,-79.00,-75.0,-74.0,-67.00,-64.0,-62.133482,-59.0,-54.0,-52.0,...,0,1,0,1,0,1,0,1,0,1
1,-76.0,-76.00,-64.0,-60.0,-57.00,-56.0,-55.255556,-54.0,-50.0,-49.0,...,1,0,0,1,1,0,0,1,0,1
2,-89.0,-80.00,-76.6,-75.0,-69.00,-63.0,-62.289089,-57.0,-54.0,-52.0,...,1,1,1,0,0,1,0,1,0,1
3,-93.0,-87.00,-83.0,-78.0,-71.00,-68.0,-67.938259,-64.0,-62.0,-61.0,...,1,1,0,1,1,0,1,0,1,0
4,-72.0,-70.00,-65.0,-63.0,-58.00,-57.0,-55.760797,-54.0,-51.0,-48.0,...,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,-87.0,-84.00,-77.0,-74.0,-67.00,-65.0,-62.998435,-58.3,-54.0,-52.0,...,0,1,0,1,0,1,0,1,0,1
15548,-93.0,-92.00,-89.0,-86.0,-81.25,-78.0,-77.020677,-74.0,-71.0,-69.0,...,1,0,1,0,0,1,1,0,0,1
15549,-99.0,-94.00,-88.0,-85.0,-80.00,-77.0,-74.469638,-71.0,-65.0,-63.0,...,1,1,0,1,1,0,0,1,1,0
15550,-88.0,-83.39,-79.0,-75.0,-71.00,-68.0,-64.321346,-60.0,-56.0,-52.0,...,1,0,1,0,0,1,1,0,0,1


In [20]:
predictors = []
for col in trainset.columns:
    if "Rssi" in col or "Coarse" in col or "_" in col:
        predictors.append(col)
#predictors

['Bluetooth:Percentile_1',
 'Bluetooth:Percentile_5',
 'Bluetooth:Percentile_10',
 'Bluetooth:Percentile_25',
 'Bluetooth:Percentile_40',
 'Bluetooth:Percentile_65',
 'Bluetooth:Percentile_80',
 'Bluetooth:Percentile_90',
 'Bluetooth:Percentile_95',
 'Bluetooth:Percentile_99',
 'Accelerometer_x:Min',
 'Accelerometer_x:Mean',
 'Accelerometer_x:Max',
 'Accelerometer_y:Min',
 'Accelerometer_y:Mean',
 'Accelerometer_y:Max',
 'Accelerometer_z:Min',
 'Accelerometer_z:Mean',
 'Accelerometer_z:Max',
 'Gyroscope_x:Min',
 'Gyroscope_x:Mean',
 'Gyroscope_x:Max',
 'Gyroscope_y:Min',
 'Gyroscope_y:Mean',
 'Gyroscope_y:Max',
 'Gyroscope_z:Min',
 'Gyroscope_z:Mean',
 'Gyroscope_z:Max',
 'Attitude_x:Min',
 'Attitude_x:Mean',
 'Attitude_x:Max',
 'Attitude_y:Min',
 'Attitude_y:Mean',
 'Attitude_y:Max',
 'Attitude_z:Min',
 'Attitude_z:Mean',
 'Attitude_z:Max',
 'Gravity_x:Min',
 'Gravity_x:Mean',
 'Gravity_x:Max',
 'Gravity_y:Min',
 'Gravity_y:Mean',
 'Gravity_y:Max',
 'Gravity_z:Min',
 'Gravity_z:Mean',

In [21]:
report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.10443474682212917

In [22]:
ndcf

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.094527,0.012882,0.107409
1,fine_grain,1.8,0.072114,0.020115,0.092229
2,fine_grain,3.0,0.038810,0.067906,0.106715
3,coarse_grain,1.8,0.088545,0.022841,0.111386


In [23]:
print(report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.96      0.91      0.93      1407
         1.8       0.94      0.89      0.91      1408
         3.0       0.90      0.95      0.92      1385
         4.5       0.89      0.93      0.91      1399

    accuracy                           0.92      5599
   macro avg       0.92      0.92      0.92      5599
weighted avg       0.92      0.92      0.92      5599



In [24]:
print(report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.98      0.91      0.94      1423
         4.5       0.92      0.98      0.95      1401

    accuracy                           0.94      2824
   macro avg       0.95      0.94      0.94      2824
weighted avg       0.95      0.94      0.94      2824



## BinaryEncoder

In [25]:
(trainset, devset, testset), pipe = make_datasets(combine(sensor_summary, read_non_sensor_data),
                                          postproc_categorical, verbose=True, testing=0,
                                          tunables={ 
                                              "CategoricalEncoder" : ce.BinaryEncoder
                                          })

100%|████████████████████████████████████████████████████████████████████████████| 15552/15552 [08:21<00:00, 30.99it/s]


NO Pipe input given!


C:\Users\mtom7\anaconda3\envs\py39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
100%|████████████████████████████████████████████████████████████████████████████████| 935/935 [00:53<00:00, 17.49it/s]


Pipe input given!


100%|██████████████████████████████████████████████████████████████████████████████| 8423/8423 [08:16<00:00, 16.97it/s]


Pipe input given!


In [26]:
#trainset

,Bluetooth:Min,Bluetooth:Percentile_1,Bluetooth:Percentile_5,Bluetooth:Percentile_10,Bluetooth:Percentile_25,Bluetooth:Percentile_40,Bluetooth:Mean,Bluetooth:Percentile_65,Bluetooth:Percentile_80,Bluetooth:Percentile_90,...,Cat:RXDevice_3,Cat:RXDevice_4,Cat:TXCarry_0,Cat:TXCarry_1,Cat:RXCarry_0,Cat:RXCarry_1,Cat:RXPose_0,Cat:RXPose_1,Cat:TXPose_0,Cat:TXPose_1
0,-88.0,-79.00,-75.0,-74.0,-67.00,-64.0,-62.133482,-59.0,-54.0,-52.0,...,0,1,0,1,0,1,0,1,0,1
1,-76.0,-76.00,-64.0,-60.0,-57.00,-56.0,-55.255556,-54.0,-50.0,-49.0,...,1,0,0,1,1,0,0,1,0,1
2,-89.0,-80.00,-76.6,-75.0,-69.00,-63.0,-62.289089,-57.0,-54.0,-52.0,...,1,1,1,0,0,1,0,1,0,1
3,-93.0,-87.00,-83.0,-78.0,-71.00,-68.0,-67.938259,-64.0,-62.0,-61.0,...,1,1,0,1,1,0,1,0,1,0
4,-72.0,-70.00,-65.0,-63.0,-58.00,-57.0,-55.760797,-54.0,-51.0,-48.0,...,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,-87.0,-84.00,-77.0,-74.0,-67.00,-65.0,-62.998435,-58.3,-54.0,-52.0,...,0,1,0,1,0,1,0,1,0,1
15548,-93.0,-92.00,-89.0,-86.0,-81.25,-78.0,-77.020677,-74.0,-71.0,-69.0,...,1,0,1,0,0,1,1,0,0,1
15549,-99.0,-94.00,-88.0,-85.0,-80.00,-77.0,-74.469638,-71.0,-65.0,-63.0,...,1,1,0,1,1,0,0,1,1,0
15550,-88.0,-83.39,-79.0,-75.0,-71.00,-68.0,-64.321346,-60.0,-56.0,-52.0,...,1,0,1,0,0,1,1,0,0,1


In [27]:
predictors = []
for col in trainset.columns:
    if "Rssi" in col or "Coarse" in col or "_" in col:
        predictors.append(col)
#predictors

['Bluetooth:Percentile_1',
 'Bluetooth:Percentile_5',
 'Bluetooth:Percentile_10',
 'Bluetooth:Percentile_25',
 'Bluetooth:Percentile_40',
 'Bluetooth:Percentile_65',
 'Bluetooth:Percentile_80',
 'Bluetooth:Percentile_90',
 'Bluetooth:Percentile_95',
 'Bluetooth:Percentile_99',
 'Accelerometer_x:Min',
 'Accelerometer_x:Mean',
 'Accelerometer_x:Max',
 'Accelerometer_y:Min',
 'Accelerometer_y:Mean',
 'Accelerometer_y:Max',
 'Accelerometer_z:Min',
 'Accelerometer_z:Mean',
 'Accelerometer_z:Max',
 'Gyroscope_x:Min',
 'Gyroscope_x:Mean',
 'Gyroscope_x:Max',
 'Gyroscope_y:Min',
 'Gyroscope_y:Mean',
 'Gyroscope_y:Max',
 'Gyroscope_z:Min',
 'Gyroscope_z:Mean',
 'Gyroscope_z:Max',
 'Attitude_x:Min',
 'Attitude_x:Mean',
 'Attitude_x:Max',
 'Attitude_y:Min',
 'Attitude_y:Mean',
 'Attitude_y:Max',
 'Attitude_z:Min',
 'Attitude_z:Mean',
 'Attitude_z:Max',
 'Gravity_x:Min',
 'Gravity_x:Mean',
 'Gravity_x:Max',
 'Gravity_y:Min',
 'Gravity_y:Mean',
 'Gravity_y:Max',
 'Gravity_z:Min',
 'Gravity_z:Mean',

In [28]:
report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.10443474682212917

In [29]:
ndcf

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.094527,0.012882,0.107409
1,fine_grain,1.8,0.072114,0.020115,0.092229
2,fine_grain,3.0,0.038810,0.067906,0.106715
3,coarse_grain,1.8,0.088545,0.022841,0.111386


In [30]:
print(report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.96      0.91      0.93      1407
         1.8       0.94      0.89      0.91      1408
         3.0       0.90      0.95      0.92      1385
         4.5       0.89      0.93      0.91      1399

    accuracy                           0.92      5599
   macro avg       0.92      0.92      0.92      5599
weighted avg       0.92      0.92      0.92      5599



In [31]:
print(report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.98      0.91      0.94      1423
         4.5       0.92      0.98      0.95      1401

    accuracy                           0.94      2824
   macro avg       0.95      0.94      0.94      2824
weighted avg       0.95      0.94      0.94      2824



## CatBoostEncoder

In [11]:
run_with_encoder(ce.CatBoostEncoder)

100%|████████████████████████████████████████████████████████████████████████████| 15552/15552 [19:31<00:00, 13.28it/s]


NO Pipe input given!


C:\Users\mtom7\anaconda3\envs\py39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
100%|████████████████████████████████████████████████████████████████████████████████| 935/935 [01:09<00:00, 13.37it/s]


Pipe input given!


100%|██████████████████████████████████████████████████████████████████████████████| 8423/8423 [10:01<00:00, 13.99it/s]


Pipe input given!
0.09265216819522434
         Subset  Distance    P_miss      P_fa      nDCF
0    fine_grain       1.2  0.086709  0.015744  0.102454
1    fine_grain       1.8  0.063588  0.019756  0.083344
2    fine_grain       3.0  0.034762  0.045032  0.079794
3  coarse_grain       1.8  0.085032  0.019986  0.105017
              precision    recall  f1-score   support

         1.2       0.95      0.91      0.93      1407
         1.8       0.93      0.89      0.91      1408
         3.0       0.92      0.95      0.93      1385
         4.5       0.90      0.95      0.93      1399

    accuracy                           0.92      5599
   macro avg       0.93      0.93      0.92      5599
weighted avg       0.93      0.92      0.92      5599

              precision    recall  f1-score   support

         1.8       0.98      0.91      0.95      1423
         4.5       0.92      0.98      0.95      1401

    accuracy                           0.95      2824
   macro avg       0.95      